main activity type code = 20 je "combustion of fuels" (spalování paliv)


In [1]:
import sys
import os

import pandas as pd

sys.path.append(os.path.abspath("..")) # ty ".." znamenají že jdu o jednu úroveň nad. A tento kus kodu musí být před tim importem "from data.analysis"

from data_analysis.allowances_utils import get_allowances_data



In [2]:
# year we want to visualize
YEAR = 2021

In [3]:
definition = [
        {'name': 'Elektrárna Počerady, a.s.',  
         'label': 'elektrárna, teplárna',
         'permit_id': ['CZ-0204-05', 'CZ-0478-12'],
         'color': 'red',
        },
        {'name': 'ArcelorMittal Ostrava a TAMEH Czech',  
         'label': 'železárna, ocelárna',
         'permit_id': ['CZ-0134-05', 'CZ-0444-10'],
         'color': 'X',
        },
        {'name': 'Elektrárna Tušimice',  
         'label': 'elektrárna, teplárna',
         'permit_id': ['CZ-0209-05'],
         'color': 'X',  
        },
        {'name': 'Elektrárna Mělník',  
         'label': 'elektrárna, teplárna',
         'permit_id': ['CZ-0202-05','CZ-0092-05','CZ-0203-05'],
         'color': 'X',  
        },    
]

## Načti si data 

In [4]:
# načti si tabulku
my_df = get_allowances_data(YEAR,"CZ",20,"../data/EUA/verified_emissions_2021_en.xlsx")

In [5]:
# UPRAV STÁVAJÍC DF
my_df.rename(columns={"value":"tCO2ekv"},inplace=True) # přejmenuj sloupec
my_df.drop("MAIN_ACTIVITY_TYPE_CODE", axis=1, inplace=True) # zahoď sloupec

In [6]:
my_df.head(3)

,tCO2ekv,IDENTIFIER_IN_REG
PERMIT_IDENTIFIER,,
CZ-0204-05,4.419807,"Elektrárna Počerady, a.s."
CZ-0201-05,3.781199,ČEZ - Elektrárna Ledvice
CZ-0207-05,3.494895,ČEZ - Elektrárna Prunéřov 2


## Uprav data

CO DĚLÁM
- vytvoř funkci "get value" - vyhodí ti hodnotu pro kod povolenky
- vytvoř nový dataframe
- iteruj přes definition a: 
    - do jednoho listu sbírej názvy podniků
    - do druhýho listu sbírej typy podniků
    - do třetího listu sbírej součty hodnot v povolenkách
- listy vlož jako sloupce do novýho df

In [7]:
def _get_value(key, df):
    return df.loc[key, 'tCO2ekv']

In [13]:
# nový data frame
test_df = pd.DataFrame()
test_df['name'] = []
test_df['value'] = []
test_df['label'] = []
test_df['squares'] = []

In [14]:
# iteruju přes definition, sčítám povolenky, vkládám do listů
facility_list = []
value_list = []
label_list = []

for facility in definition:
    facility_list.append(facility['name'])
    label_list.append(facility['label'])
    total = 0
    for permit_code in facility["permit_id"]:
        local_value = _get_value(permit_code, my_df)
        total += local_value
    value_list.append(total)

In [26]:
# vkládám listy do df
test_df['name'] = facility_list
test_df['value'] = value_list
test_df['label'] = label_list

# dopočítej čtverečky
test_df['squares'] = (test_df['value'] / 0.25).round()

# úprava df
test_df.set_index('name', inplace = True)
test_df.sort_values('value', ascending = False, inplace = True)
test_df

,value,label,squares
name,,,
"Elektrárna Počerady, a.s.",5.389507,"elektrárna, teplárna",22.0
ArcelorMittal Ostrava a TAMEH Czech,5.124220,"železárna, ocelárna",20.0
Elektrárna Tušimice,3.399410,"elektrárna, teplárna",14.0
Elektrárna Mělník,2.390403,"elektrárna, teplárna",10.0


Dotazy na Honzu:
- může takle vypadat ta definiční struktura?
- v excelové tabulce "Emise Česká republika" jsou kody povolenek (třeba CZ-0231-05), které nejsou v tab. "verified_emissions_2021_en". Co s tím? 

Pozn:
- vim, že to pak budu dávat do funkcí. Zatim takle.

In [ ]:
# ČEK: najdi povolenku
print(my_df.loc['CZ-0203-05', 'tCO2ekv'])

# PRO PROCES UČENÍ SE - Nebude ve finálním kodu

## Iteruju přes řádky 

In [ ]:
# novej df
new_df = pd.DataFrame() 

In [ ]:
# iteruj přes řádku my_df, každý přidej do new_df
n = 0
for value in my_df.index: # pro hodnotu v my_df.index (tedy: CZ-0204-05, CZ-0201-05 atd.)
    new_df = new_df.append(my_df.iloc[[n]]) # ulož n-tý řádek (tedy "my_df.iloc[[n]]") do nového df ("new_df")
    n += 1 # skoč na další řádek ("zvětči n o 1")
new_df

In [ ]:
# iteruj přes řádku my_df, podle podmínky přidej řádek do new_df
practice_df = pd.DataFrame()
n = 0
for value in my_df.index: # pro hodnotu v my_df.index (tedy: CZ-0204-05, CZ-0201-05 atd.)
    if my_df.index[n] in ["CZ-0204-05", "CZ-0228-05"]: # pokud je hodnota indexu v množině... pak... 
        practice_df = practice_df.append(my_df.iloc[[n]]) # ulož n-tý řádek (tedy "my_df.iloc[[n]]") do nového df ("new_df")
    n += 1 # skoč na další řádek ("zvětči n o 1")
practice_df

In [ ]:
# iteruj přes řádky df ve sloupici "IDENTIFIER_IN_REG" a pokud hodnota = .... udělej něco...
for value in my_df["IDENTIFIER_IN_REG"]:
    if value == "Elektrárna Počerady, a.s.":
        print("a")
    else:
        print("c")

In [ ]:
# iteruj přes řádky df ve sloupici "PERMIT_IDENTIFIER" (tedy "index") a pro každou hodnotu napiš o jedna se zvětšující číslo
n = 0
for value in my_df.index:
    n += 1
    print(n)

In [ ]:
# iteruj přes řádky df ve sloupici "PERMIT_IDENTIFIER" (tedy "index") a pokud hodnota = .... udělej něco... 
for value in my_df.index:
    if value == "CZ-0478-12":
        print("a")
    elif value == "CZ-0204-05":
        print("b")
    else:
        print("c")

In [ ]:
# iteruj přes řádku my_df, každý přidej do new_df
n = 0
for value in my_df.index: # pro hodnotu v my_df.index (tedy: CZ-0204-05, CZ-0201-05 atd.)
    new_df = new_df.append(my_df.iloc[[n]]) # ulož n-tý řádek (tedy "my_df.iloc[[n]]") do nového df ("new_df")
    n += 1 # skoč na další řádek ("zvětči n o 1")
new_df

## Další zajímavé kousky kodu 

In [ ]:
# Elektrárna Počerady: součet
A = my_df.loc['CZ-0478-12', 'tCO2ekv']
B = my_df.loc['CZ-0204-05', 'tCO2ekv']
print(A+B)

In [ ]:
# dostaň se k tomu "permit_id"
for facility in definition:
    a = facility["permit_id"]
    print(a)

In [ ]:
# dostaň se k jednotlivým složkám toho "permit_id"
for facility in definition:
    a = facility["permit_id"]
    for member in a:
        print(member)

In [ ]:
# sečti přes for cyklus
total = 0
for value in my_df["tCO2ekv"]:
    total += value
total 

In [ ]:
# sečti přes funkci
total2 = my_df["tCO2ekv"].sum()
total2

In [ ]:
# ukaž mi n-tý řádek z my_df
n = 1
my_df.iloc[[n]]

## POZNATKY

- iloc[] lokalizuje podle čísla indexu